In [17]:

import os
import openai
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import pandas as pd
import matplotlib.pyplot as plt


import time
from queue import Queue
from threading import Thread
import openai


In [31]:
OPENAI_API_KEY = ##KEY
os.environ["OPENAI_API_KEY"] = ##KEY
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
gpt4_ratings = pd.read_excel("/Users/mahathi/Dissertation/ratings_data/total_gpt4.xlsx")
gpt4_ratings

,proc_text,conversation_id,id,int_mean,int_var,rating
0,teacher: hello <student>! student: hi <teacher>!,118,22313,0.333333,0.333333,INT: 1\nEXP_INT: 3\nREASON: The conversation i...
1,teacher: hi - welcome to the chat how are you?...,118,22314,2.666667,1.333333,INT: 3 EXP_INT: 3 REASON: The teacher's welcom...
2,"teacher: oh austria, interesting! all the past...",118,22315,1.666667,1.333333,INT: 3\nEXP_INT: 3\nREASON: The teacher's comm...
3,"teacher: i'm from ukraine, but i've lived in t...",118,22316,2.666667,2.333333,INT: 3 \nEXP_INT: 3 \nREASON: This conversatio...
4,"teacher: yes, i was an english teacher and cam...",118,22317,2.666667,2.333333,INT: 4\nEXP_INT: 4\nREASON: The teacher's resp...
...,...,...,...,...,...,...
6271,"student: yes as a result of, young people real...",777,67122,2.000000,1.000000,NaN
6272,"student: yes, that all teacher: ok thanks hol...",777,67123,2.666667,5.333333,NaN
6273,student: it is difficult for children to disti...,777,67124,1.666667,2.333333,NaN
6274,"student: okey, i saw it...i think adult is may...",777,67125,1.333333,0.333333,NaN


In [9]:
gpt_int = []
for idx, i in enumerate(gpt4_ratings['rating']):
    if isinstance(i, str):
        if len(i) > 1 and 'INT:' in i:
            temp = i.split('INT:')[1].strip()[0]
        elif len(i) > 1 and 'INT :' in i:
            temp = i.split('INT :')[1].strip()[0]
        else:
            print(idx, i)
            temp = "None"
        gpt_int.append(temp)
    else:
        # print(f"Skipping index {idx} with non-string value: {i}")
        gpt_int.append("None")

In [10]:
gpt4_ratings['gpt_int'] = gpt_int
gpt4_ratings

,proc_text,conversation_id,id,int_mean,int_var,rating,gpt_int
0,teacher: hello <student>! student: hi <teacher>!,118,22313,0.333333,0.333333,INT: 1\nEXP_INT: 3\nREASON: The conversation i...,1
1,teacher: hi - welcome to the chat how are you?...,118,22314,2.666667,1.333333,INT: 3 EXP_INT: 3 REASON: The teacher's welcom...,3
2,"teacher: oh austria, interesting! all the past...",118,22315,1.666667,1.333333,INT: 3\nEXP_INT: 3\nREASON: The teacher's comm...,3
3,"teacher: i'm from ukraine, but i've lived in t...",118,22316,2.666667,2.333333,INT: 3 \nEXP_INT: 3 \nREASON: This conversatio...,3
4,"teacher: yes, i was an english teacher and cam...",118,22317,2.666667,2.333333,INT: 4\nEXP_INT: 4\nREASON: The teacher's resp...,4
...,...,...,...,...,...,...,...
6271,"student: yes as a result of, young people real...",777,67122,2.000000,1.000000,NaN,None
6272,"student: yes, that all teacher: ok thanks hol...",777,67123,2.666667,5.333333,NaN,None
6273,student: it is difficult for children to disti...,777,67124,1.666667,2.333333,NaN,None
6274,"student: okey, i saw it...i think adult is may...",777,67125,1.333333,0.333333,NaN,None


In [11]:
gpt4_id_list = gpt4_ratings['id'].unique()
gpt4_id_list

array([22313, 22314, 22315, ..., 67124, 67125, 67126])

In [3]:
conv_data = pd.read_excel("/Users/mahathi/Dissertation/LLM_Analysis/final_data/complete_annotation_data.xlsx")
avg_vals = conv_data[['text','conversation_id','id','human_int']].groupby(['text','conversation_id','id',],
                                                                     ).agg(human_int_mean= ('human_int', 'mean'), 
                                                                           human_int_var= ('human_int', 'var'),
                                                                           ).reset_index()
avg_vals

,text,conversation_id,id,human_int_mean,human_int_var
0,student: teacher: does it snow a lot in ukra...,92,58405,1.333333,0.333333
1,student: teacher: has he been to the usa? i ...,92,58407,2.666667,0.333333
2,"student: teacher: ok <student>, excellent wo...",92,58410,2.333333,1.333333
3,student: ! sure! teacher: well done for learn...,131,35877,1.666667,1.333333
4,student: (*/*) i think i haven't teacher: som...,103,39257,1.666667,0.333333
...,...,...,...,...,...
6651,teacher: you've guessed the chamber of secrets...,103,65616,0.666667,0.333333
6652,teacher: your bookshop sounds like a lovely pl...,154,45838,1.333333,0.333333
6653,teacher: your classmates? you still don't have...,125,59667,1.333333,0.333333
6654,teacher: your noun 'production' is a good word...,20,64387,3.333333,0.333333


In [24]:
avg_vals1 = avg_vals.sort_values('id').reset_index(drop=True)
avg_vals1

,text,conversation_id,id,human_int_mean,human_int_var
0,teacher: hello <student>! student: hi <teacher>!,118,22313,0.333333,0.333333
1,teacher: hi - welcome to the chat how are you?...,118,22314,2.666667,1.333333
2,"teacher: oh austria, interesting! all the past...",118,22315,1.666667,1.333333
3,"teacher: i'm from ukraine, but i've lived in t...",118,22316,2.666667,2.333333
4,"teacher: yes, i was an english teacher and cam...",118,22317,2.666667,2.333333
...,...,...,...,...,...
6651,student: let me think past simple! teacher: o...,52,67110,1.333333,0.333333
6652,student: because in recent year is present per...,52,67111,2.333333,1.333333
6653,student: yes.. i think i can understand it now...,52,67112,1.333333,2.333333
6654,student: yes that is what i like to do teache...,52,67113,1.000000,0.000000


In [25]:
gpt4_ratings[gpt4_ratings['id'].isin(avg_vals1['id'].unique())]

,proc_text,conversation_id,id,int_mean,int_var,rating,gpt_int
0,teacher: hello <student>! student: hi <teacher>!,118,22313,0.333333,0.333333,INT: 1\nEXP_INT: 3\nREASON: The conversation i...,1
1,teacher: hi - welcome to the chat how are you?...,118,22314,2.666667,1.333333,INT: 3 EXP_INT: 3 REASON: The teacher's welcom...,3
2,"teacher: oh austria, interesting! all the past...",118,22315,1.666667,1.333333,INT: 3\nEXP_INT: 3\nREASON: The teacher's comm...,3
3,"teacher: i'm from ukraine, but i've lived in t...",118,22316,2.666667,2.333333,INT: 3 \nEXP_INT: 3 \nREASON: This conversatio...,3
4,"teacher: yes, i was an english teacher and cam...",118,22317,2.666667,2.333333,INT: 4\nEXP_INT: 4\nREASON: The teacher's resp...,4
...,...,...,...,...,...,...,...
6260,student: let me think past simple! teacher: o...,52,67110,1.333333,0.333333,NaN,None
6261,student: because in recent year is present per...,52,67111,2.333333,1.333333,NaN,None
6262,student: yes.. i think i can understand it now...,52,67112,1.333333,2.333333,NaN,None
6263,student: yes that is what i like to do teache...,52,67113,1.000000,0.000000,NaN,None


In [26]:
prompts_to_run = avg_vals1[~avg_vals1['id'].isin(gpt4_id_list)]
prompts_to_run

,text,conversation_id,id,human_int_mean,human_int_var
3823,"student: hi, <teacher>! how is life? studen...",214,58291,2.000000,1.000000
3824,"teacher: hiya! all good, thank you! how are yo...",214,58292,3.333333,1.333333
3825,teacher: have you had a nice yoga session? yes...,214,58293,2.666667,0.333333
3826,teacher: what about wednesday? student: and s...,214,58294,1.666667,0.333333
3827,teacher: oh i see! fair enough student: today...,214,58295,2.000000,3.000000
...,...,...,...,...,...
6041,"teacher: yes, good! what about equality'? it's...",167,66077,2.333333,0.333333
6042,"teacher: ok, i'll give you a hint two plus two...",167,66078,2.333333,0.333333
6043,teacher: if men and women have the same rights...,167,66079,2.000000,0.000000
6044,"teacher: they are not the same, but they are e...",167,66080,1.666667,0.333333


In [35]:
prompts_to_run

,text,conversation_id,id,human_int_mean,human_int_var,first_role,second_role
3823,"student: hi, <teacher>! how is life? studen...",214,58291,2.000000,1.000000,Student,Teacher
3824,"teacher: hiya! all good, thank you! how are yo...",214,58292,3.333333,1.333333,Teacher,Student
3825,teacher: have you had a nice yoga session? yes...,214,58293,2.666667,0.333333,Teacher,Student
3826,teacher: what about wednesday? student: and s...,214,58294,1.666667,0.333333,Teacher,Student
3827,teacher: oh i see! fair enough student: today...,214,58295,2.000000,3.000000,Teacher,Student
...,...,...,...,...,...,...,...
6041,"teacher: yes, good! what about equality'? it's...",167,66077,2.333333,0.333333,Teacher,Student
6042,"teacher: ok, i'll give you a hint two plus two...",167,66078,2.333333,0.333333,Teacher,Student
6043,teacher: if men and women have the same rights...,167,66079,2.000000,0.000000,Teacher,Student
6044,"teacher: they are not the same, but they are e...",167,66080,1.666667,0.333333,Teacher,Student


In [27]:
st_rate_msg = ["Hello! Can I please ask you to rate a student teacher conversation based on how interesting they feel? Could you please provide a rating between 0 to 4. O being boring. 4 being most interesting. It is a conversation between a student (2nd language English) learning English from a teacher. Assume the role of the student based on the conversation. Please give the Rating as an integer value in this format: INT : 'how interesting you find the reply of the teacher'. Please keep the previous conversation in mind. The conversation is: "]
te_rate_msg = ["Hello! Can I please ask you to rate a student teacher conversation based on how interesting they feel? Could you please provide a rating between 0 to 4. O being boring. 4 being most interesting. It is a conversation between a student (2nd language English) learning English from a teacher. Assume the role of the teacher based on the conversation. Please give the Rating as an integer value in this format: INT : 'how interesting you find the reply of the student'. Please keep the previous conversation in mind. The conversation is: "]


prompts_to_run['first_role'] = prompts_to_run['text'].apply(lambda x : "Teacher" if x.startswith('teacher') else 'Student')
prompts_to_run['second_role'] = prompts_to_run['text'].apply(lambda x : "Student" if x.startswith('teacher') else 'Teacher')


convs_prompts = []
prev_conv_id = 0

for idx, con in prompts_to_run.iterrows():
    
    conv = con['text']
    first_role = con['first_role']
    second_role = con['second_role']

    if (first_role=='Teacher'):
        prompt = te_rate_msg[0] + conv
 
    else:
        prompt = st_rate_msg[0] + conv
   


    convs_prompts.append(prompt)


/var/folders/y0/kj7xts31345_t0dd0vr2szgr0000gn/T/ipykernel_21697/3801620485.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompts_to_run['first_role'] = prompts_to_run['text'].apply(lambda x : "Teacher" if x.startswith('teacher') else 'Student')
/var/folders/y0/kj7xts31345_t0dd0vr2szgr0000gn/T/ipykernel_21697/3801620485.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompts_to_run['second_role'] = prompts_to_run['text'].apply(lambda x : "Student" if x.startswith('teacher') else 'Teacher')


In [42]:
convs_prompts[0]

"Hello! Can I please ask you to rate a student teacher conversation based on how interesting they feel? Could you please provide a rating between 0 to 4. O being boring. 4 being most interesting. It is a conversation between a student (2nd language English) learning English from a teacher. Assume the role of the student based on the conversation. Please give the Rating as an integer value in this format: INT : 'how interesting you find the reply of the teacher'. Please keep the previous conversation in mind. The conversation is: student: hi, <teacher>! how is life?    student: hello, <teacher>. how are you?"

In [43]:
storing_list = []

In [50]:
storing_list2 = []

In [52]:
# Set up OpenAI API client
openai.api_key = ##KEY

# Create a queue for requests
request_queue = Queue()

# Define the rate limit and delay parameters
rate_limit = 10 # Number of requests allowed per minute
delay = 60 / rate_limit  # Delay between each request

# Worker function to process requests
def process_requests():
    while True:
        message = request_queue.get()
        make_api_request(message)
        request_queue.task_done()

# Function to make an API request
def make_api_request(message):
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": message}]
    )
    storing_list2.append( completion.choices[0].message['content'])
    print("CHATGPT : ", completion.choices[0].message['content'])

In [53]:

# Start worker threads
num_threads = min(rate_limit, 20)
for _ in range(num_threads):
    worker = Thread(target=process_requests)
    worker.daemon = True
    worker.start()



i=0
for message in convs_prompts[1928:]:
    time.sleep(delay)
    print(i, " MY MESSAGE: ", message)
    request_queue.put(message)
    print("--------------------")
    i+=1

    if i%300==0:
        time.sleep(30)

     

# Wait until all requests are processed
request_queue.join()


0  MY MESSAGE:  Hello! Can I please ask you to rate a student teacher conversation based on how interesting they feel? Could you please provide a rating between 0 to 4. O being boring. 4 being most interesting. It is a conversation between a student (2nd language English) learning English from a teacher. Assume the role of the teacher based on the conversation. Please give the Rating as an integer value in this format: INT : 'how interesting you find the reply of the student'. Please keep the previous conversation in mind. The conversation is: teacher: oh good! you would have been correct anyway :) so are there any subjects where you never cheat?  student: and it was easy to cheat on history o don't cheat on math because like it doesn't even make any sense ypu just need to know how to make it    student: and it was easy to cheat on history i don't cheat on math because it doesn't require much effort you just need to know how to solve it
--------------------
CHATGPT :  3 : 'The student'

In [56]:
gpt4_missing  = storing_list + storing_list2


In [59]:
prompts_to_run['gpt4_rating'] = gpt4_missing
prompts_to_run.to_excel('gpt4_missing.xlsx', index=False)

/var/folders/y0/kj7xts31345_t0dd0vr2szgr0000gn/T/ipykernel_21697/538258783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prompts_to_run['gpt4_rating'] = gpt4_missing
